In [ ]:
from datasets import concatenate_datasets, Dataset, DatasetDict
import evaluate
from huggingface_hub import notebook_login, HfFolder
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import wandb

In [2]:
df= pd.read_csv('final_master_cleaned.csv')

In [ ]:
df.head(3)

In [3]:
!git config --global user.email "email id"
!git config --global user.name "username"

In [ ]:
wandb.login()

%env WANDB_PROJECT=Flan-T5-master

In [ ]:
notebook_login() #hugging face login

In [9]:
#Data split 70% train-20% test-10% valid
train, test = train_test_split(df, test_size=0.2,random_state=42)
train, valid = train_test_split(train, test_size=0.125,random_state=42)

In [10]:
raw_train = Dataset.from_pandas(train)
raw_test = Dataset.from_pandas(test)

In [11]:
raw_train=raw_train.remove_columns('__index_level_0__')
raw_test = raw_test.remove_columns('__index_level_0__')

In [12]:
ds_dict = {'train' : raw_train
    ,'test' : raw_test}

dataset = DatasetDict(ds_dict)
dataset

In [14]:
valid.to_csv('test_master.csv')#held out data used for evaluation later

In [15]:
model_id="google/flan-t5-base"
# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# The maximum total input sequence length after tokenization.
#also trucate and pad sequences
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["context"], truncation=True), batched=True, remove_columns=["context", "question"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["question"], truncation=True), batched=True, remove_columns=["context", "question"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")


In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["Generate question for this text: " + item for item in sample["context"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["question"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["context", "question", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")


In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


In [20]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [21]:
# HuggingFace repository id
repository_id = f"{model_id.split('/')[1]}-master-final"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=f"new/{repository_id}",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16=False,
    learning_rate=5e-5,
    num_train_epochs=2,
    # logging & evaluation strategies
    logging_dir=f"new/{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # push to hub parameters
    report_to="wandb",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)


In [ ]:
# Start training
trainer.train()


In [ ]:
trainer.evaluate()

In [ ]:
wandb.finish()

In [ ]:
tokenizer.save_pretrained(f"new/{repository_id}")
model.push_to_hub("flan-t5-base-master-final")
tokenizer.push_to_hub("flan-t5-base-master-final")

Evaluation


In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("prnv13/flan-t5-base-master")
model = AutoModelForSeq2SeqLM.from_pretrained("prnv13/flan-t5-base-master")

In [8]:
def run_model(input_string, **generator_args):

  generator_args = {
  "max_length": 512,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = "generate question for this text: " + input_string
  input_ids = tokenizer.encode(input_string, return_tensors="pt")
  res = model.generate(input_ids, **generator_args)
  output = tokenizer.batch_decode(res, skip_special_tokens=True)
  return output[0]

In [9]:
df = pd.read_csv('test-master.csv') #held out data from goldstandard while finetuning the model. 10%

In [10]:
predicted = []
for c in df["context"]:
    predicted.append(run_model(c))

convert model results and questions to the required format for eval

In [ ]:
model_results = []
for i, sentence in enumerate(predicted, start=1):
    model_results.append((i, [sentence]))

In [16]:
references = []
for i, sentence in enumerate(df["question"], start=1):
    references.append((i, [sentence]))

In [ ]:
from src.evaluation.eval_main import Metrics

metrics = Metrics(save_to_file=True)

result = pd.DataFrame(
    metrics.evaluate(model_output=model_results, references=references),
    index=["T5-master_final"]
)
result

In [ ]:
result.to_csv('result.csv')

In [19]:
import matplotlib.pyplot as plt
result.plot(kind="bar", figsize=(15,10), title="Example Plot showing all Metrics all at once") # Just for demo purposes. Would not recommend!
plt.show()